## Перенесем все полученные функции в первом задании


In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce


def KNN_Imputer(X: pd.DataFrame, with_return_col_category= False):
    print("Всего пропусков было:")
    temp_columns_where_nan_value = X.isna().sum()
    print(temp_columns_where_nan_value[temp_columns_where_nan_value > 0])
    
    mask = ~(X.dtypes == np.object_)
    imputer = KNNImputer(n_neighbors= 5,
                         weights= 'distance',
                         add_indicator= True) # Показывает, где раньше были пропуски
    
    col_name = ['is_changed' + el for el in mask.index[temp_columns_where_nan_value > 0].to_list()]# столбцы по которым были совершены изменения / вставки значений

    temp_df = pd.DataFrame(data= imputer.fit_transform(X.T[mask].T),
                           columns= mask[mask == True].index.to_list() + col_name,
                           index= X.index)

    if with_return_col_category:
        return pd.concat([temp_df, X.T[~mask].T], axis= 1, join= 'inner'), mask[~(mask == True)].index

    
    return pd.concat([temp_df, X.T[~mask].T], axis= 1, join= 'inner')

def category_encoding(X, cols, encoder = 'one_hot', y= None):
    match encoder:

        case 'WOE':
            incoder = ce.WOEEncoder(cols= cols)
            return incoder.fit_transform(X, y)
        
        case 'one_hot':
            return pd.concat([X.drop(cols, axis=1), pd.get_dummies(X[cols])], axis=1)

In [17]:
data = pd.read_csv("donations.csv", index_col= 1)
# data = data.drop(index= data[data['TargetB'] == 0].index)
X= data.drop(columns= ['TargetB', 'TargetD'])
y= data[['TargetB', 'TargetD']]

y['TargetD'] = SimpleImputer(fill_value= 0, strategy= 'constant').fit_transform(y[['TargetD']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

X_train, cat = KNN_Imputer(X_train, with_return_col_category= True)
X_test = KNN_Imputer(X_test)

X_train = category_encoding(X_train, cat, y= y_train)
X_test = category_encoding(X_test, cat, y= y_test)

/tmp/ipykernel_5689/1925555179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['TargetD'] = SimpleImputer(fill_value= 0, strategy= 'constant').fit_transform(y[['TargetD']])


Всего пропусков было:
GiftAvgCard36    1263
DemAge           1682
DemMedIncome     1625
dtype: int64
Всего пропусков было:
GiftAvgCard36    517
DemAge           725
DemMedIncome     732
dtype: int64


In [18]:
## Реализуем свой градиентный бустинг: 
import numpy as np
from sklearn.tree import DecisionTreeRegressor

class GBCustomClassifier:
    """
    ОСНОВНЫЕ ОТЛИЧИЯ от регрессионной модели:
        - нужно преобразовывать y: onehotencoding (если много классов) /
          приводить к вероятностному виду используя сигмоиду
        - использование logloss

    Условие:
        - Для бинарной классификации использовать logloss.
        - Нужно поддержать все параметры, перечисленные в __init__.
        - Нужно написать реализацию всех перечисленных в теле класса методов.
        - В качестве реализации дерева решения нужно использовать DecisionTreeRegressor.
        - Другими классами из sklearn пользоваться запрещается.
    """

    def __init__(
        self,
        *,
        learning_rate=0.1,
        n_estimators=100,
        criterion="friedman_mse",
        min_samples_split=2,
        min_samples_leaf=1,
        max_depth=3,
        random_state=None
    ):
        """
        Args:
            learning_rate (float): Скорость обучения или по-другому шаг, как в градиентном спуске.
                Defaults to 0.1.
            n_estimators (int): Количество базовых алгоритмов. Defaults to 100.
            criterion (str): отвечает за критерий сплита в дереве. Defaults to "friedman_mse".
            min_samples_split (int): Узел не должен делиться, если в нем объем выборке равен или меньше
                чем min_samples_split. Defaults to 2.
            min_samples_leaf (int): Именно в листьях должно быть не меньше чем min_samples_leaf выборок.
                Defaults to 1.
            max_depth (int): Максимальная глубина дерева. Defaults to 3.
            random_state (int): Начальная точка для рандомных чисел. Defaults to None.
        """
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []

    @staticmethod
    def sigmoid(y: np.array):
        """
        Формула: 1 / (1 + exp(-x))
        """
        return 1 / (1 + np.exp(-y))

    def fit(self, x, y):
        y_mean = np.mean(y)
        self.base = np.log(y_mean / (1 - y_mean)) if y_mean not in (0, 1) else 0
        y_pred = np.full_like(y, self.base, dtype=float)
        for _ in range(self.n_estimators):
            p = self.sigmoid(y_pred)
            residuals = y - p
            temp_tree = DecisionTreeRegressor(
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_depth=self.max_depth,
                random_state=self.random_state,
                criterion=self.criterion
            )
            temp_tree.fit(x, residuals, sample_weight=p * (1 - p))
            self.trees.append(temp_tree)
            y_pred += self.learning_rate * temp_tree.predict(x)

    def predict_proba(self, x):
        logit = self.base + self.learning_rate * np.sum(
            [tree.predict(x) for tree in self.trees], axis=0
        )
        proba_pos = self.sigmoid(logit)
        proba_neg = 1 - proba_pos
        return np.vstack((proba_neg, proba_pos)).T

    def predict(self, x):
        pred = (self.predict_proba(x) >= 0.5).astype(int)
        return pred[:, 1]

    @property
    def estimators_(self):
        return self.trees

In [20]:
from sklearn.metrics import log_loss, f1_score
model = GBCustomClassifier()
model.fit(X_train, y_train['TargetB'])
pr_prob = model.predict_proba(X_test)
print(f1_score(y_test['TargetB'], model.predict(X_test)), log_loss(y_test['TargetB'], pr_prob))

0.536777897380696 0.6814817824949682


In [21]:
test_data = pd.read_csv('test.csv', index_col= 0)
test_data, cat = KNN_Imputer(test_data, with_return_col_category= True)
test_data= category_encoding(test_data, cat)

y_labels = model.predict(test_data)
y_labels.sum() * 0.68

Всего пропусков было:
GiftAvgCard36    20526
GiftTimeFirst        2
DemAge           24260
dtype: int64


KeyboardInterrupt: 

In [ ]:
test_data

,GiftCnt36,GiftCntAll,GiftCntCard36,GiftCntCardAll,GiftAvgLast,GiftAvg36,GiftAvgAll,GiftAvgCard36,GiftTimeLast,GiftTimeFirst,...,PromCntCardAll,StatusCat96NK,StatusCatStarAll,DemCluster,DemAge,DemGender,DemHomeOwner,DemMedHomeValue,DemPctVeterans,DemMedIncome
ID,,,,,,,,,,,,,,,,,,,,,
45361,3,8,2,4,4.0,9.00,7.25,9.0,18,44.0,...,11,A,0,0,NaN,M,U,0,0,0
14972,1,2,1,2,20.0,20.00,15.00,20.0,25,39.0,...,9,A,0,0,NaN,F,U,0,0,0
14975,3,8,3,6,15.0,11.00,8.50,11.0,16,53.0,...,18,A,1,0,NaN,U,U,0,0,0
187614,4,7,3,6,20.0,20.00,19.29,20.0,16,41.0,...,7,E,1,0,NaN,F,U,0,0,0
33265,7,10,6,8,10.0,8.71,8.20,8.5,21,45.0,...,17,S,1,35,38.0,M,U,0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185006,2,3,0,1,20.0,15.00,11.67,NaN,18,39.0,...,13,A,0,42,NaN,U,U,111000,25,0
185005,3,5,1,2,10.0,10.67,9.40,7.0,18,37.0,...,13,A,0,40,NaN,F,U,121000,35,59813
185001,1,2,1,1,50.0,50.00,32.50,50.0,23,23.0,...,9,N,0,42,31.0,M,H,111000,25,59813
